In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
#ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135544
Azure region: southcentralus
Subscription id: 48a74bb7-9950-4cc1-9caa-5d50f995cc55
Resource group: aml-quickstarts-135544


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
cpu_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-01-21T04:11:47.916000+00:00', 'errors': None, 'creationTime': '2021-01-21T03:15:55.203267+00:00', 'modifiedTime': '2021-01-21T03:16:11.472284+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [13]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,normal
from azureml.core import ScriptRunConfig
from azureml.core import Environment
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        'learning_rate': normal(10,3),
        'keep_probability': uniform(0.05,0.1)
    }
)

# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#if "training" not in os.listdir():
#    os.mkdir("./training")
project_folder = './'


#sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

# Create a SKLearn estimator for use with train.py
compute_target = ws.compute_targets['auto-ml']
est = SKLearn(source_directory=project_folder,
            entry_script='train.py',
            compute_target=compute_target,
            #arguments=['--kernel', 'linear', '--penalty', 1.0],
            #environment=sklearn_env)
            )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                                    hyperparameter_sampling=ps,
                                    policy=early_termination_policy,
                                    primary_metric_name='Accuracy', 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                    max_total_runs=4,
                                    max_concurrent_runs=4)

In [14]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_50ad1a1a-3ce4-4709-9f91-11e0cc2d5cc1
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_50ad1a1a-3ce4-4709-9f91-11e0cc2d5cc1?wsid=/subscriptions/48a74bb7-9950-4cc1-9caa-5d50f995cc55/resourcegroups/aml-quickstarts-135544/workspaces/quick-starts-ws-135544

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-21T04:00:54.970359][API][INFO]Experiment created<END>\n"<START>[2021-01-21T04:00:56.0261175Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-21T04:00:55.919091][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-21T04:00:56.255035][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_50ad1a1a-3ce4-4709-9f91-11e0cc2d5cc1
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_50ad1a1a-3ce4-4709-9f91-11e0cc2d5cc1?wsid=/subscriptions/48a74

In [15]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
model = best_run.register_model(model_name='hyperdrive-train', model_path='outputs/model.joblib')

['--keep_probability', '0.07825463651646461', '--learning_rate', '9.107501477055617']
['azureml-logs/55_azureml-execution-tvmps_d19252b8e45e75640ac873f5a8f270923f49c0899033fd240d0fc709699561b8_d.txt', 'azureml-logs/65_job_prep-tvmps_d19252b8e45e75640ac873f5a8f270923f49c0899033fd240d0fc709699561b8_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_d19252b8e45e75640ac873f5a8f270923f49c0899033fd240d0fc709699561b8_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/101_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [4]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y

In [5]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="regression",
    primary_metric='normalized_root_mean_squared_error',
    training_data=x,
    label_column_name='y',
    n_cross_validations=5)

In [6]:
# Submit your automl run

exp = Experiment(ws,"automl_run")
automl_run = exp.submit(config=automl_config, show_output = True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_48ae85ce-f461-4247-89c5-a38cbf976d78

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***************************************************

In [7]:
# Retrieve and save your best automl model.

import joblib
# Get your best run and save the model from that run.

best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
#model = best_run.register_model(model_name='automl-train', model_path='outputs/model.joblib')

AttributeError: 'AutoMLRun' object has no attribute 'get_best_run_by_primary_metric'